In [38]:
import os.path
from imutils import paths, resize
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense

import pickle

In [40]:
def get_letters_list2(im_name):
    image = cv2.imread(im_name,0)
    image = cv2.bitwise_not(image)
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=4)
    sizes = stats[1:, -1]
    nb_components = nb_components - 1
    min_size = 100
    for i in range(0, nb_components):
        if sizes[i] <= min_size:
            image[output == i + 1] = 0
    edge = 16
    inv_cropped = image[edge:image.shape[0]-edge,edge:image.shape[1]-edge]
    new_image, letters_only_contour, hierarchy = cv2.findContours(inv_cropped,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    letters_list = []
    cropped = cv2.bitwise_not(inv_cropped)
    # sorts contours by x position
    starting_points = []
    for cnt in letters_only_contour:
        x,y,w,h = cv2.boundingRect(cnt)
        starting_points.append(x)
    sorted_letters_only_contour = [x for _,x in sorted(zip(starting_points,letters_only_contour))]
    # append letters to list.  following code detects unually long letters, and assumes the contour found multiple letters
    # splits such contours in half until it's a reasonable length
    for cnt in sorted_letters_only_contour:
        x,y,w,h = cv2.boundingRect(cnt)
        temp_width = w
        while temp_width > 34:
            if temp_width < 60:
                temp_width = np.floor_divide(temp_width,2)
            elif temp_width < 91:
                temp_width = np.floor_divide(temp_width,3)
            else:
                temp_width = np.floor_divide(temp_width,2)
        for i in range(np.floor_divide(w,temp_width)):
            letters_list.append(cropped[y:y+h,x+i*temp_width:x+(i+1)*temp_width])
    while len(letters_list)<6:
        letter_widths = []
        for letter in letters_list:
            letter_widths.append(letter.shape[1])
        widest_letter_index = letter_widths.index(max(letter_widths))
        widest_letter = letters_list[widest_letter_index]
        letters_list.remove(widest_letter)
        letters_list.insert(widest_letter_index,widest_letter[:,np.floor_divide(widest_letter.shape[1],2):])
        letters_list.insert(widest_letter_index,widest_letter[:,0:np.floor_divide(widest_letter.shape[1],2)])
    if len(letters_list)>6:
        return ('An error has occurred.  Too many letters detected',letters_list)
    else:
        return letters_list

In [41]:
def normalize_image_size(image):
    width = 20
    height = 20
    (h,w) = image.shape[:2]
    if w > h:
        image = resize(image, width=width)
    else:
        image = resize(image, height=height)
    padW = int((width-image.shape[1])/2.0)
    padH = int((height-image.shape[0])/2.0)
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,
        cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))
    return image

In [12]:
paths.list_images('grainy_samples/letters')

<generator object list_files at 0x000002141F92BE08>

In [19]:
data = []
labels = []
count = 0
for letter_path in paths.list_images('grainy_samples/letters'):
    image = cv2.imread(letter_path,0)
    
    # Resize image
    image = normalize_image_size(image)

    
    image = np.expand_dims(image, axis = 2)
    data.append(image)
    labels.append(letter_path.split(os.path.sep)[-2])
    count+=1
    if count%2000==0:
        print('Added image #%d'%count)


Added image #2000
Added image #4000
Added image #6000
Added image #8000
Added image #10000
Added image #12000
Added image #14000
Added image #16000
Added image #18000
Added image #20000
Added image #22000
Added image #24000
Added image #26000
Added image #28000
Added image #30000
Added image #32000
Added image #34000
Added image #36000
Added image #38000
Added image #40000
Added image #42000
Added image #44000
Added image #46000
Added image #48000
Added image #50000
Added image #52000
Added image #54000
Added image #56000
Added image #58000
Added image #60000
Added image #62000
Added image #64000
Added image #66000
Added image #68000
Added image #70000
Added image #72000
Added image #74000
Added image #76000
Added image #78000
Added image #80000
Added image #82000
Added image #84000
Added image #86000
Added image #88000
Added image #90000
Added image #92000
Added image #94000


In [21]:
data = np.array(data,dtype='float')/255.0
labels = np.array(labels)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(data,labels, test_size=0.25)

In [24]:
lb = LabelBinarizer().fit(y_train)
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

In [27]:
model = Sequential()

In [29]:
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

In [30]:
model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

In [32]:
model.add(Flatten())
model.add(Dense(500, activation="relu"))

In [34]:
model.add(Dense(19, activation="softmax"))

In [35]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [36]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=19, epochs=10, verbose=1)

Train on 71238 samples, validate on 23746 samples
Epoch 1/10
71238/71238 [==============================] - 177s 2ms/step - loss: 0.1620 - acc: 0.9571 - val_loss: 0.0582 - val_acc: 0.9859
Epoch 2/10
71238/71238 [==============================] - 176s 2ms/step - loss: 0.0465 - acc: 0.9877 - val_loss: 0.0587 - val_acc: 0.9864
Epoch 3/10
71238/71238 [==============================] - 176s 2ms/step - loss: 0.0315 - acc: 0.9912 - val_loss: 0.0532 - val_acc: 0.9877
Epoch 4/10
71238/71238 [==============================] - 172s 2ms/step - loss: 0.0220 - acc: 0.9938 - val_loss: 0.0536 - val_acc: 0.9877
Epoch 5/10
71238/71238 [==============================] - 174s 2ms/step - loss: 0.0167 - acc: 0.9951 - val_loss: 0.0630 - val_acc: 0.9869
Epoch 6/10
71238/71238 [==============================] - 178s 2ms/step - loss: 0.0138 - acc: 0.9962 - val_loss: 0.0679 - val_acc: 0.9872
Epoch 7/10
71238/71238 [==============================] - 176s 2ms/step - loss: 0.0116 - acc: 0.9968 - val_loss: 0.0688 - 

In [37]:
model.save('10Epoch_model.h5')

In [39]:
with open('model_label_map.dat','wb') as f:
    pickle.dump(lb,f)

In [42]:
test = get_letters_list2('test_CAPTCHA.png')
attempted_password = ''
for let in test:
    let = normalize_image_size(let)
    let = np.expand_dims(let,axis=2)
    let = np.expand_dims(let,axis=0)
    prediction = model.predict(let)
    letter_output = lb.inverse_transform(prediction)[0]
    attempted_password+=letter_output

In [43]:
attempted_password

'wvphnh'